In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from imblearn.over_sampling import RandomOverSampler
import xgboost as xgb
from imblearn.combine import SMOTEENN


In [19]:
df = pd.read_csv(r"C:\Users\Afnan Khan\Desktop\New folder\All_DF.csv")

In [20]:
df["Amt_vs_AvgRatio"] = df["Transaction_Amount"] / (df["Avg_Amt_1h"] + 1e-6)
df["Velocity_IP_Interaction"] = df["Velocity_Score"] * df["IP_Risk_Score"]
df["Txn_Foreign_Ratio"] = df["Txn_Count_5m"] / (df["Foreign_Country_Count_1d"] + 1e-6)


In [21]:
df.head(1)

,TransactionID,UserID,Transaction_Amount,IP_Risk_Score,VPN_Flag,Final_Outcome,Geo_Match,Txn_Count_5m,Avg_Amt_1h,Foreign_Country_Count_1d,Velocity_Score,Old_Status,New_Status,Change_Reason,Amt_vs_AvgRatio,Velocity_IP_Interaction,Txn_Foreign_Ratio
0,TID_000001,UID_01855,262.53,3.5,True,CLEARED,True,1.0,14.49,1.0,18.2,PWD_RESET,CLOSED,FRAUD_BLOCK,18.118011,63.7,0.999999


In [22]:
df.drop(['TransactionID', 'UserID', 'Old_Status', 'New_Status',	'Change_Reason', 'Foreign_Country_Count_1d', 'Txn_Count_5m'], axis=1, inplace=True)

In [23]:
label_encoder = LabelEncoder()

df['VPN_Flag'] = label_encoder.fit_transform(df['VPN_Flag'])
df['Final_Outcome'] = label_encoder.fit_transform(df['Final_Outcome'])
df['Geo_Match'] = label_encoder.fit_transform(df['Geo_Match'])


In [24]:
df.head(1)

,Transaction_Amount,IP_Risk_Score,VPN_Flag,Final_Outcome,Geo_Match,Avg_Amt_1h,Velocity_Score,Amt_vs_AvgRatio,Velocity_IP_Interaction,Txn_Foreign_Ratio
0,262.53,3.5,1,0,1,14.49,18.2,18.118011,63.7,0.999999


In [25]:
x = df.drop(columns=['Final_Outcome'])
y = df['Final_Outcome']

In [26]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)


In [27]:
x_train = x_train.values
x_train

array([[1.27240000e+02, 3.50000000e+00, 0.00000000e+00, ...,
        7.24601325e+00, 8.54000000e+01, 0.00000000e+00],
       [1.57280000e+02, 9.80000000e+00, 1.00000000e+00, ...,
        1.57547830e+00, 1.37200000e+02, 0.00000000e+00],
       [4.03570000e+02, 2.70000000e+00, 0.00000000e+00, ...,
        1.08355484e+00, 6.42600000e+01, 1.00000000e+06],
       ...,
       [1.34646000e+03, 5.33000000e+01, 0.00000000e+00, ...,
        2.67473177e+01, 0.00000000e+00, 0.00000000e+00],
       [6.81950000e+02, 1.04000000e+01, 0.00000000e+00, ...,
        8.02860833e+00, 1.56000000e+02, 0.00000000e+00],
       [1.00279000e+03, 4.60000000e+00, 0.00000000e+00, ...,
        4.22779205e+00, 8.74000000e+01, 1.00000000e+06]])

In [28]:
x_test = x_test.values
x_test

array([[9.87900000e+01, 1.00000000e+00, 0.00000000e+00, ...,
        2.63159290e+00, 1.08000000e+01, 1.00000000e+06],
       [4.31770000e+02, 7.90000000e+00, 0.00000000e+00, ...,
        2.90499897e+00, 2.68600000e+02, 0.00000000e+00],
       [1.86330000e+02, 1.13000000e+01, 0.00000000e+00, ...,
        2.66719149e+00, 1.24300000e+02, 0.00000000e+00],
       ...,
       [7.38000000e+01, 6.00000000e+00, 0.00000000e+00, ...,
        7.38000000e+07, 6.00000000e+00, 0.00000000e+00],
       [2.80680000e+02, 2.07000000e+01, 0.00000000e+00, ...,
        4.31815318e+01, 4.55400000e+01, 0.00000000e+00],
       [3.89170000e+02, 9.00000000e+00, 0.00000000e+00, ...,
        9.04836064e+00, 1.17000000e+01, 0.00000000e+00]])

In [29]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)   
x_test = scaler.transform(x_test)         


In [ ]:
# sme = SMOTEENN(random_state=42)
# x_train_res, y_train_res = sme.fit_resample(x_train, y_train)

# print("Before:", y_train.value_counts(normalize=True))
# print("After:", y_train_res.value_counts(normalize=True))


In [13]:
import xgboost as xgb

model = xgb.XGBClassifier(
    objective='binary:logistic',
    n_estimators=800,            
    learning_rate=0.018,         
    max_depth=9,                 
    min_child_weight=1,          
    subsample=0.85,
    colsample_bytree=0.8,
    gamma=1,
    reg_lambda=2,
    reg_alpha=1,
    scale_pos_weight=(len(y_train[y_train==0]) / len(y_train[y_train==1])) * 1.0,  
    eval_metric='auc',
    random_state=42,
    base_score=0.5 
)


In [14]:
model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.018, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=800, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [15]:
train_acc = model.score(x_train, y_train)
test_acc = model.score(x_test, y_test)

print("Train Accuracy:", train_acc)
print("Test Accuracy:", test_acc)

Train Accuracy: 0.7342509289440656
Test Accuracy: 0.730431499715165


In [ ]:
y_pred_proba = model.predict_proba(x_test)[:, 1]
y_pred = (y_pred_proba > 0.59).astype(int)
 
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("AUC Score:", roc_auc_score(y_test, y_pred_proba))


In [ ]:
******************************************

In [31]:
from sklearn.ensemble import RandomForestClassifier

RC = RandomForestClassifier(
    n_estimators=800,          # trees ki sankhya
    max_depth=9,               # har tree ki max depth
    min_samples_split=2,       # node split hone ke liye min samples
    min_samples_leaf=1,        # leaf node me min samples
    max_features='sqrt',       # har split par random features
    bootstrap=True,            # bootstrapping on
    class_weight='balanced',   # imbalance handle karega
    random_state=42,           # reproducibility
    n_jobs=-1                  # saare CPU cores use kare speed ke liye
)


In [ ]:
RC.fit(x_train, y_train)

In [ ]:
train_acc = model.score(x_train, y_train)
test_acc = model.score(x_test, y_test)

print("Train Accuracy:", train_acc)
print("Test Accuracy:", test_acc)

In [ ]:
y_pred_proba = model.predict_proba(x_test)[:, 1]
y_pred = (y_pred_proba > 0.59).astype(int)
 
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("AUC Score:", roc_auc_score(y_test, y_pred_proba))


# 2. Use XAI tool SHAP, jo har transaction ke liye risk score ke saath top “reason codes” (features) bhi batata hai, taaki regulator aur analyst samajh saken kyun system ne us transaction ko risky declare kiya

In [32]:
import shap


In [36]:
# 1️⃣ Sample data lo
x_sample = pd.DataFrame(x_test[:20], columns=df.columns.drop('Final_Outcome'))


In [37]:
explainer = shap.Explainer(model.predict, x_sample)
shap_values = explainer(x_sample)

In [38]:
# 3️⃣ SHAP values ko DataFrame me convert karo (feature names automatic le lega)
reason_codes = pd.DataFrame(
    shap_values.values,
    columns=x_sample.columns
)

In [39]:
# 4️⃣ Risk score calculate karo aur add karo
reason_codes["Risk_Score"] = np.abs(shap_values.values).mean(axis=1)
reason_codes.head(4)

,Transaction_Amount,IP_Risk_Score,VPN_Flag,Geo_Match,Avg_Amt_1h,Velocity_Score,Amt_vs_AvgRatio,Velocity_IP_Interaction,Txn_Foreign_Ratio,Risk_Score
0,0.356885,0.018730,0.052004,0.130873,-0.035258,0.039980,0.107897,-0.043710,0.022599,0.089771
1,0.150476,0.352262,0.024405,-0.015119,0.055357,-0.090595,0.104881,0.071786,-0.003452,0.096481
2,-0.180238,-0.099048,0.010476,-0.022560,-0.000238,-0.014643,0.015952,-0.060357,0.000655,0.044907
3,-0.198452,-0.131964,0.022381,-0.019643,-0.001250,-0.012798,0.007976,-0.001726,-0.014524,0.045635


In [43]:
# 1️⃣ Risk Score calculate karo
reason_codes["Risk_Score"] = np.abs(shap_values.values).mean(axis=1)

# 2️⃣ Top 3 impactful features (reason codes) nikal lo
top_features = np.argsort(-np.abs(shap_values.values), axis=1)[:, :3]
top_reasons = []
for i in range(len(x_sample)):
    features = x_sample.columns[top_features[i]].tolist()
    top_reasons.append(", ".join(features))
reason_codes["Top_Reason_Codes"] = top_reasons

# 3️⃣ Risk level assign karo based on Risk_Score
conditions = [
    reason_codes["Risk_Score"] < 0.05,            # Low risk
    (reason_codes["Risk_Score"] >= 0.05) & (reason_codes["Risk_Score"] < 0.10),  # Medium
    reason_codes["Risk_Score"] >= 0.10            # High risk
]
choices = ["Low", "Medium", "High"]
reason_codes["Risk_Level"] = np.select(conditions, choices, default="Low")

# 4️⃣ Sirf important columns dikhao
reason_codes[["Risk_Score", "Risk_Level", "Top_Reason_Codes"]].head(5)


,Risk_Score,Risk_Level,Top_Reason_Codes
0,0.089771,Medium,"Transaction_Amount, Geo_Match, Amt_vs_AvgRatio"
1,0.096481,Medium,"IP_Risk_Score, Transaction_Amount, Amt_vs_AvgR..."
2,0.044907,Low,"Transaction_Amount, IP_Risk_Score, Velocity_IP..."
3,0.045635,Low,"Transaction_Amount, IP_Risk_Score, VPN_Flag"
4,0.064484,Medium,"VPN_Flag, Amt_vs_AvgRatio, Transaction_Amount"


In [45]:
import joblib
joblib.dump(model, "fraud_model.pkl")


['fraud_model.pkl']

In [40]:
df.head(1)

,Transaction_Amount,IP_Risk_Score,VPN_Flag,Final_Outcome,Geo_Match,Avg_Amt_1h,Velocity_Score,Amt_vs_AvgRatio,Velocity_IP_Interaction,Txn_Foreign_Ratio
0,262.53,3.5,1,0,1,14.49,18.2,18.118011,63.7,0.999999
